In [23]:
from transformers import BartForConditionalGeneration, BartTokenizer
from datasets import load_dataset
import pandas as pd
from sklearn.model_selection import train_test_split
from datasets import Dataset
from transformers import Trainer, TrainingArguments
from transformers import Seq2SeqTrainingArguments
import torch

In [2]:
# Load the pre-trained BART model and tokenizer
model = BartForConditionalGeneration.from_pretrained('facebook/bart-large-cnn')
tokenizer = BartTokenizer.from_pretrained('facebook/bart-large-cnn')

config.json:   0%|          | 0.00/1.58k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.63G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/363 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

In [3]:
dataset = load_dataset('cnn_dailymail', '3.0.0')

Generating train split:   0%|          | 0/287113 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/13368 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/11490 [00:00<?, ? examples/s]

In [4]:
dataset

DatasetDict({
    train: Dataset({
        features: ['article', 'highlights', 'id'],
        num_rows: 287113
    })
    validation: Dataset({
        features: ['article', 'highlights', 'id'],
        num_rows: 13368
    })
    test: Dataset({
        features: ['article', 'highlights', 'id'],
        num_rows: 11490
    })
})

In [5]:
train_data = dataset['train']
train_df = train_data.to_pandas()
val_data = dataset['validation']
val_df = train_data.to_pandas()
test_data = dataset['test']
test_df = train_data.to_pandas()

In [6]:
df = train_df.sample(10)

In [7]:
df = df.drop('id', axis='columns')

In [8]:
df = df.dropna()

In [9]:
train_df, test_df = train_test_split(df, test_size=0.1, random_state=42)

In [10]:
train_df = train_df.reset_index(drop=True)
test_df = test_df.reset_index(drop=True)

In [11]:
train_dataset = Dataset.from_pandas(train_df)
test_dataset = Dataset.from_pandas(test_df)

In [12]:
# from datasets import Features, ClassLabel, Value

# # Define features
# features = Features({
#     'article': Value('string'),
#     'highlights': Value('string')
# })

# # Cast the dataset to enforce these features
# hf_dataset = hf_dataset.cast(features)

In [13]:
train_dataset

Dataset({
    features: ['article', 'highlights'],
    num_rows: 9
})

In [14]:
def preprocess_data(examples):
    inputs = tokenizer(examples['article'], max_length=1024, truncation=True, padding="max_length")
    outputs = tokenizer(examples['highlights'], max_length=128, truncation=True, padding="max_length")

    inputs['labels'] = outputs.input_ids
    return inputs

In [15]:
# def preprocess_data(examples):
#     model_inputs = tokenizer(examples['article'], max_length=1024, truncation=True)

#     # Setup the tokenizer for targets
#     with tokenizer.as_target_tokenizer():
#         labels = tokenizer(examples['highlights'], max_length=128, truncation=True)

#     model_inputs['labels'] = labels['input_ids']
#     return model_inputs

In [16]:
train_token_dataset = train_dataset.map(preprocess_data, batched=True)
test_token_dataset = test_dataset.map(preprocess_data, batched=True)

Map:   0%|          | 0/9 [00:00<?, ? examples/s]

Map:   0%|          | 0/1 [00:00<?, ? examples/s]

In [17]:
# training_args = TrainingArguments(
#     output_dir='./results',
#     evaluation_strategy='epoch',
#     learning_rate=5e-5,
#     per_device_train_batch_size=4,
#     per_device_eval_batch_size=4,
#     weight_decay=0.01,
#     save_total_limit=3,
#     num_train_epochs=1,
#     predict_with_generate=True
# )

training_args = Seq2SeqTrainingArguments(
    output_dir='./results',
    evaluation_strategy='epoch',
    learning_rate=5e-5,
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    weight_decay=0.01,
    save_total_limit=3,
    num_train_epochs=1,
    logging_dir='./logs',
    predict_with_generate=True
)

In [18]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_token_dataset,
    eval_dataset=test_token_dataset,
    tokenizer=tokenizer
)

In [19]:
trainer.train()

Epoch,Training Loss,Validation Loss
1,No log,6.682439


TrainOutput(global_step=3, training_loss=5.783547083536784, metrics={'train_runtime': 97.3004, 'train_samples_per_second': 0.092, 'train_steps_per_second': 0.031, 'total_flos': 19503941419008.0, 'train_loss': 5.783547083536784, 'epoch': 1.0})

In [ ]:
model.save_pretrained('./bart_summarization_model')
tokenizer.save_pretrained('./bart_summarization_model')

In [20]:
text = "Your input text for summarization goes here. This text should ideally represent the type of content you trained your model on, such as news articles for a model trained on the CNN/DailyMail dataset."
inputs = tokenizer(text, return_tensors="pt", max_length=1024, truncation=True, padding="max_length")

In [24]:
model.to("cuda")
inputs = {key: value.to("cuda") for key, value in inputs.items()}

model.eval()

# Generate summary
with torch.no_grad():
    summary_ids = model.generate(inputs['input_ids'], attention_mask=inputs['attention_mask'],
                                 max_length=150, num_beams=5, early_stopping=True)

# Decode generated summary
summary = tokenizer.decode(summary_ids[0], skip_special_tokens=True)

In [25]:
print(summary)

Your input text for summarization goes here. This text should ideally represent the type of content you trained your model on, such as news articles for a model trained on the CNN/DailyMail dataset. For more information on how to use this data, visit CNN iReport.


In [ ]:
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer, Seq2SeqTrainer, Seq2SeqTrainingArguments
from datasets import load_dataset

# Load the dataset
dataset = load_dataset('cnn_dailymail', '3.0.0')

# Load a pre-trained model and tokenizer suitable for text summarization
model_checkpoint = 'bert-base-uncased'  # Example checkpoint, replace with your model
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)
model = AutoModelForSeq2SeqLM.from_pretrained(model_checkpoint)

# Tokenize the dataset
def preprocess_function(examples):
    return tokenizer(examples['article'], truncation=True, padding='max_length', max_length=512)

tokenized_datasets = dataset.map(preprocess_function, batched=True)

# Define training arguments
training_args = Seq2SeqTrainingArguments(
    output_dir='./results',
    evaluation_strategy='epoch',
    learning_rate=1e-4,
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    num_train_epochs=3,
    save_steps=500,
    save_total_limit=3,
    logging_dir='./logs',
    predict_with_generate=True,
)

# Create a Seq2SeqTrainer
trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets['train'],
    eval_dataset=tokenized_datasets['validation'],
)

# Fine-tune the model
trainer.train()

In [ ]:
from transformers import GPT2Tokenizer, GPT2LMHeadModel, GPT2Config, Trainer, TrainingArguments
from datasets import load_dataset

# Load the dataset
dataset = load_dataset('cnn_dailymail', '3.0.0')

# Load a pre-trained GPT-2 model and tokenizer
model_name = "gpt2"  # Example model name, replace with specific GPT model (e.g., "gpt2-medium", "gpt2-large")
tokenizer = GPT2Tokenizer.from_pretrained(model_name)
model = GPT2LMHeadModel.from_pretrained(model_name)

# Tokenize the dataset
def preprocess_function(examples):
    return tokenizer(examples['article'], truncation=True, padding='max_length', max_length=512)

tokenized_datasets = dataset.map(preprocess_function, batched=True)

# Define training arguments
training_args = TrainingArguments(
    output_dir='./results',
    evaluation_strategy='epoch',
    learning_rate=5e-5,
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    num_train_epochs=3,
    save_steps=500,
    save_total_limit=3,
    logging_dir='./logs',
    predict_with_generate=True,
)

# Create a Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets['train'],
    eval_dataset=tokenized_datasets['validation'],
)

# Fine-tune the model
trainer.train()

In [ ]:
from transformers import MarianTokenizer, MarianMTModel, Trainer, TrainingArguments
from datasets import load_dataset

# Load the dataset
dataset = load_dataset('wmt16', 'de-en')

# Load a pre-trained MarianMT model and tokenizer for English to German translation
model_name = 'Helsinki-NLP/opus-mt-en-de'
tokenizer = MarianTokenizer.from_pretrained(model_name)
model = MarianMTModel.from_pretrained(model_name)

# Tokenize the dataset
def preprocess_function(examples):
    return tokenizer(examples['translation']['en'], truncation=True, padding='max_length', max_length=512)

tokenized_datasets = dataset.map(preprocess_function, batched=True)

# Define training arguments
training_args = TrainingArguments(
    output_dir='./results',
    evaluation_strategy='epoch',
    learning_rate=1e-4,
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    num_train_epochs=3,
    save_steps=500,
    save_total_limit=3,
    logging_dir='./logs',
)

# Create a Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets['train'],
    eval_dataset=tokenized_datasets['validation'],
)

# Fine-tune the model
trainer.train()

In [ ]:
from transformers import GPT2Tokenizer, GPT2LMHeadModel, Trainer, TrainingArguments
from datasets import load_dataset

# Load the dataset
dataset = load_dataset('wmt16', 'de-en')

# Load a pre-trained GPT-2 model and tokenizer
model_name = 'gpt2'
tokenizer = GPT2Tokenizer.from_pretrained(model_name)
model = GPT2LMHeadModel.from_pretrained(model_name)

# Tokenize the dataset
def preprocess_function(examples):
    source_text = examples['translation']['de']
    target_text = examples['translation']['en']
    return tokenizer(source_text, text_pair=target_text, truncation=True, padding='max_length', max_length=512)

tokenized_datasets = dataset.map(preprocess_function, batched=True)

# Define training arguments
training_args = TrainingArguments(
    output_dir='./results',
    evaluation_strategy='epoch',
    learning_rate=1e-4,
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    num_train_epochs=3,
    save_steps=500,
    save_total_limit=3,
    logging_dir='./logs',
)

# Create a Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets['train'],
    eval_dataset=tokenized_datasets['validation'],
)

# Fine-tune the model
trainer.train()